## Problem Statement

In this assignment students need to predict whether a person makes over 50K per year
or not from classic adult dataset using XGBoost.

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)
test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', skiprows = 1, header = None)
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 
                'occupation','relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week',
                 'native_country', 'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

In [3]:
train_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
train_set.shape

(32561, 15)

## EDA

In [5]:
(train_set["native_country"].value_counts() / train_set.shape[0]).head()

 United-States    0.895857
 Mexico           0.019748
 ?                0.017905
 Philippines      0.006081
 Germany          0.004207
Name: native_country, dtype: float64

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
train_set.groupby("workclass").mean()['hours_per_week']

workclass
 ?                   31.919390
 Federal-gov         41.379167
 Local-gov           40.982800
 Never-worked        28.428571
 Private             40.267096
 Self-emp-inc        48.818100
 Self-emp-not-inc    44.421881
 State-gov           39.031587
 Without-pay         32.714286
Name: hours_per_week, dtype: float64

In [7]:
train_set.groupby(["wage_class",'sex']).count()

age  workclass  fnlwgt  education  education_num  \
wage_class sex                                                           
 <=50K      Female   9592       9592    9592       9592           9592   
            Male    15128      15128   15128      15128          15128   
 >50K       Female   1179       1179    1179       1179           1179   
            Male     6662       6662    6662       6662           6662   

                    marital_status  occupation  relationship   race  \
wage_class sex                                                        
 <=50K      Female            9592        9592          9592   9592   
            Male             15128       15128         15128  15128   
 >50K       Female            1179        1179          1179   1179   
            Male              6662        6662          6662   6662   

                    capital_gain  capital_loss  hours_per_week  native_country  
wage_class sex                                                                  
 <=50K      Female          9592          9592            9592            9592  
            Male           15128         15128           15128           15128  
 >50K       Female          1179          1179            1179            1179  
            Male            6662          6662            6662            6662

In [8]:
train_set.groupby(['education',"wage_class"]).count()

age  workclass  fnlwgt  education_num  \
education     wage_class                                           
 10th          <=50K       871        871     871            871   
               >50K         62         62      62             62   
 11th          <=50K      1115       1115    1115           1115   
               >50K         60         60      60             60   
 12th          <=50K       400        400     400            400   
               >50K         33         33      33             33   
 1st-4th       <=50K       162        162     162            162   
               >50K          6          6       6              6   
 5th-6th       <=50K       317        317     317            317   
               >50K         16         16      16             16   
 7th-8th       <=50K       606        606     606            606   
               >50K         40         40      40             40   
 9th           <=50K       487        487     487            487   
               >50K         27         27      27             27   
 Assoc-acdm    <=50K       802        802     802            802   
               >50K        265        265     265            265   
 Assoc-voc     <=50K      1021       1021    1021           1021   
               >50K        361        361     361            361   
 Bachelors     <=50K      3134       3134    3134           3134   
               >50K       2221       2221    2221           2221   
 Doctorate     <=50K       107        107     107            107   
               >50K        306        306     306            306   
 HS-grad       <=50K      8826       8826    8826           8826   
               >50K       1675       1675    1675           1675   
 Masters       <=50K       764        764     764            764   
               >50K        959        959     959            959   
 Preschool     <=50K        51         51      51             51   
 Prof-school   <=50K       153        153     153            153   
               >50K        423        423     423            423   
 Some-college  <=50K      5904       5904    5904           5904   
               >50K       1387       1387    1387           1387   

                          marital_status  occupation  relationship  race  \
education     wage_class                                                   
 10th          <=50K                 871         871           871   871   
               >50K                   62          62            62    62   
 11th          <=50K                1115        1115          1115  1115   
               >50K                   60          60            60    60   
 12th          <=50K                 400         400           400   400   
               >50K                   33          33            33    33   
 1st-4th       <=50K                 162         162           162   162   
               >50K                    6           6             6     6   
 5th-6th       <=50K                 317         317           317   317   
               >50K                   16          16            16    16   
 7th-8th       <=50K                 606         606           606   606   
               >50K                   40          40            40    40   
 9th           <=50K                 487         487           487   487   
               >50K                   27          27            27    27   
 Assoc-acdm    <=50K                 802         802           802   802   
               >50K                  265         265           265   265   
 Assoc-voc     <=50K                1021        1021          1021  1021   
               >50K                  361         361           361   361   
 Bachelors     <=50K                3134        3134          3134  3134   
               >50K                 2221        2221          2221  2221   
 Doctorate     <=50K                 107         107           107   107   
               >50K               

In [9]:
train_set["isTrain"] = 1

In [10]:
test_set["isTrain"] = 0

In [11]:
com_data= pd.concat([train_set,test_set])

In [12]:
com_data.fillna("unknown",inplace=True)

In [13]:
data=com_data.replace({' ?':"unknown"})

In [14]:
data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
wage_class        0
isTrain           0
dtype: int64

In [15]:
data1=pd.get_dummies(data=data,columns=["workclass","education","marital_status","occupation","relationship","race","sex","native_country"])

In [16]:
data1.head()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,wage_class,isTrain,workclass_ Federal-gov,workclass_ Local-gov,...,native_country_ Puerto-Rico,native_country_ Scotland,native_country_ South,native_country_ Taiwan,native_country_ Thailand,native_country_ Trinadad&Tobago,native_country_ United-States,native_country_ Vietnam,native_country_ Yugoslavia,native_country_unknown
0,39,77516,13,2174,0,40,<=50K,1,0,0,...,0,0,0,0,0,0,1,0,0,0
1,50,83311,13,0,0,13,<=50K,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,38,215646,9,0,0,40,<=50K,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,53,234721,7,0,0,40,<=50K,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,28,338409,13,0,0,40,<=50K,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
data1=data1.replace({' <=50K':0," >50K":1,' <=50K.':0," >50K.":1})

In [18]:
train_set = data1[data1["isTrain"]==1]
test_set =  data1[data1["isTrain"]==0]

train_set.drop("isTrain",axis=1,inplace=True)
test_set.drop("isTrain",axis=1,inplace=True)

/home/skull/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [19]:
target=pd.DataFrame(train_set["wage_class"])
train_set.drop("wage_class",axis=1,inplace=True)

output=pd.DataFrame(test_set["wage_class"])
test_set.drop("wage_class",axis=1,inplace=True)


/home/skull/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## building model

In [20]:
import xgboost as xgb

In [21]:
dtrain = xgb.DMatrix(train_set.values, target.values)
dtest = xgb.DMatrix(test_set.values)

In [22]:
print("Train dataset contains {0} rows and {1} columns".format(dtrain.num_row(), dtrain.num_col()))
print("Test dataset contains {0} rows and {1} columns".format(dtest.num_row(), dtest.num_col()))

Train dataset contains 32561 rows and 108 columns
Test dataset contains 16281 rows and 108 columns


In [23]:
params = {
    'objective':'binary:logistic',
    'max_depth':2,
    'silent':1,
    'eta':1
}

num_rounds = 5

## Training classifier

In [24]:
bst = xgb.train(params, dtrain, num_rounds)

## predictions

In [25]:
preds_prob = bst.predict(dtest)


In [26]:
out = np.array(output.values)
labels = dtest.get_label()
preds = preds_prob > 0.5 # threshold
correct = 0

for i in range(len(preds)):
    if (out[i] == preds[i]):
        correct += 1

print('Predicted correctly: {0}/{1}'.format(correct, len(preds)))
print('Error: {0:.4f}'.format(1-correct/len(preds)))

Predicted correctly: 13839/16281
Error: 0.1500
